In [1]:
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import pickle
from tensorflow.keras.models import load_model

In [2]:
## Load the trained model, scaler pickle, one hot pickle
model = load_model('model.h5')


## load the encoder and scaler
with open('label_enc_gender.pkl', 'rb') as file:
    label_enc_gender = pickle.load(file)

with open('OHE.pkl', 'rb') as file:
    OHE = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)



In [3]:
# Example input data 

input_data = {
    'CreditScore': 600,
    'Gender':'Male', 
    'Age':40,
    'Tenure':3, 
    'Balance':60000, 
    'NumOfProducts': 2,
    'HasCrCard':1, 
    'IsActiveMember':1, 
    'EstimatedSalary':50000, 
    'Geography':'France'
}

In [4]:
geo_encoded = OHE.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=OHE.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Codes\Practice\Deep Learning\DL Project - ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [ ]:

input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,600,Male,40,3,60000,2,1,1,50000,France


In [11]:
input_df['Gender'] = label_enc_gender.transform(input_df['Gender'])

ValueError: y contains previously unseen labels: 1

In [12]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,600,1,40,3,60000,2,1,1,50000,France


In [13]:
# concatination with OHE data
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
## Scaling the inputs

input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
## Prediction
prediction = model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


In [17]:
prediction

array([[0.02570065]], dtype=float32)

In [19]:
prediction_proba = prediction[0][0]

In [20]:
if prediction_proba > 0.5:
    print("Likely to churn")
else:
    print("Not likely to churn")

Not likely to churn
